In [1]:
import numpy as np
from numpy import genfromtxt
from collections import defaultdict
import csv
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
pd.set_option("display.precision", 1)
import pickle


In [2]:
items = pd.read_csv("Data/items.csv")
comments = pd.read_csv("Data/comments.csv")
items

C:\Users\salma\AppData\Local\Temp\ipykernel_15084\3504774399.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  comments = pd.read_csv("Data/comments.csv")


,productname,productdescription,rating,sellerid,price,stock,image,category
0,"obat pembasmi pohon dan akar, racun pohon, sta...",Nufarm Starlon Herbisida Sistemik\n\nIsi : 100...,4.9,she-basmi,Rp38.000,96653,9a6345e2-6ae6-4a44-b7cd-042a0a0e6f8b,Pestisida dan Obat tanaman
1,100ml Pestisida Organik Neem Oil /Minyak Mimba...,Takaran 1 sendok utk 1 liter larutan siap paka...,4.9,Panda Farm,Rp25.500,91,58835cb1-a00a-437b-9132-35f538e6bae1,Pestisida dan Obat tanaman
2,Antracol Fungisida-Obat Jamur Tanaman 70WP-250gr,Isi : 250gr\nExp : 01/25\n\nAntracol adalah fu...,4.9,benihkita,Rp49.550,33,3603111_2f32cce9-11e1-4106-8c35-ce50f50f69f3_7...,Pestisida dan Obat tanaman
3,"DESTHIN obat hama tanaman, ulat, kutu putih, h...","kemasan 500ml\n\nsudah menggunakan spray baru,...",4.9,she-basmi,Rp35.000,95969,e6dc4ec2-d4be-4565-b2db-8e51b1624dc2,Pestisida dan Obat tanaman
4,"herbisida obat pembasmi rumput liar, gulma, al...","Kemasan botol 500ml\n\nbotol rusak, bocor, spr...",4.9,she-basmi,Rp35.000,95886,618a27c5-a666-4d96-91b0-29f7d74fcc11,Pestisida dan Obat tanaman
...,...,...,...,...,...,...,...,...
144,ISKU Pressure Sprayer 1L/2L Botol Semprotan Ta...,ãTerima kasih atas kepercayaan dan dukungan ...,4.9,Isku Official Shop,3989000000,4250,id-11134207-7r98q-lvd27q0a7htx37,Alat dan Mesin Pertanian
145,ISKU Semprotan Rumput Sprayer 5L Dan Botol Sem...,ãTerima kasih atas kepercayaan dan dukungan ...,5.0,Isku Official Shop,12499000000,2439,id-11134207-7r98s-lwffeqkx4o3hb9,Alat dan Mesin Pertanian
146,[2PCS] ISKU Alat Semprot Tanaman 5L Dan Botol ...,ãTerima kasih atas kepercayaan dan dukungan ...,4.9,Isku Official Shop,15899000000,2284,id-11134207-7r98y-lwffeqkwdyqy6b,Alat dan Mesin Pertanian
147,ISKU Gunting Rumput Taman Gunting tanaman gunt...,ãTerima kasih atas kepercayaan dan dukungan ...,4.9,Isku Official Shop,9989000000,7909,sg-11134201-23010-4w2r3464n4lv74,Alat dan Mesin Pertanian


In [3]:
comments

,productid,productname,comment,rating,ratingangka,userid,user_id,category
0,1,"herbisida obat pembasmi rumput liar, gulma, al...","produknya bagus. maka-nya aku beli lagi, nih....",bintang 5,5,1,Bambang,Pestisida & Obat Pertanian
1,1,"herbisida obat pembasmi rumput liar, gulma, al...",AKU SUKA PRODUKNYA. MANJUR...!!!,bintang 5,5,1,Bambang,Pestisida & Obat Pertanian
2,1,"herbisida obat pembasmi rumput liar, gulma, al...",Sudah sampai. Belum sempat dicoba. Semoga sesu...,bintang 5,5,1,Bambang,Pestisida & Obat Pertanian
3,5,"obat pembasmi pohon dan akar, racun pohon, sta...",Packing rapih dan sesuai deskripsi,bintang 5,5,1,Bambang,Pestisida & Obat Pertanian
4,5,"obat pembasmi pohon dan akar, racun pohon, sta...","Barang sudah diterima, kurir nya cepat",bintang 5,5,1,Bambang,Pestisida & Obat Pertanian
...,...,...,...,...,...,...,...,...
174891,1,"herbisida obat pembasmi rumput liar, gulma, al...",NaN,2,2,8117,4qv00hrohw,Pestisida & Obat Pertanian
174892,143,INFARM - Pupuk Organik Cair Tanaman Buah / Bun...,NaN,2,2,8117,4qv00hrohw,Pupuk dan Nutrisi Tanaman
174893,132,Pupuk Cair Daun Super Spray Siap Pakai by BLOO...,NaN,3,3,8117,4qv00hrohw,Pupuk dan Nutrisi Tanaman
174894,12,Mangga Harum Manis (1kg) Sayurbox,NaN,2,2,8117,4qv00hrohw,Produk Pertanian Segar


In [4]:
def load_data():
    ''' called to load preprepared data for the lab '''
    item_train = genfromtxt('Data/item_train.csv', delimiter=',')
    user_train = genfromtxt('Data/user_train.csv', delimiter=',')
    y_train    = genfromtxt('Data/y_train.csv', delimiter=',')
    with open('Data/item_train_header.txt', newline='') as f:    #csv reader handles quoted strings better
        item_features = list(csv.reader(f))[0]
    with open('Data/user_train_header.txt', newline='') as f:
        user_features = list(csv.reader(f))[0]
    item_vecs = genfromtxt('Data/item_train.csv', delimiter=',')

    movie_dict = defaultdict(dict)
    count = 0
#    with open('./data/movies.csv', newline='') as csvfile:
    with open('Data/product_id.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for line in reader:
            if count == 0:
                count += 1  #skip header
                #print(line) print
            else:
                count += 1
                movie_id = int(line[0])
                movie_dict[movie_id]["productname"] = line[1]
                movie_dict[movie_id]["category"] = line[2]

    return(item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict)

In [5]:
# Load Data, set configuration variables
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict = load_data()

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 174896


In [6]:
def pprint_train(x_train, features, vs, u_s, maxcount=5, user=True):
    """ Prints user_train or item_train nicely """
    if user:
        flist = [".0f", ".0f", ".1f",
                 ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f", ".1f"]
    else:
        flist = [".0f", ".0f", ".1f",
                 ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f", ".0f"]

    head = features[:vs]
    if vs < u_s: print("error, vector start {vs} should be greater then user start {u_s}")
    for i in range(u_s):
        head[i] = "[" + head[i] + "]"
    genres = features[vs:]
    hdr = head + genres
    disp = [split_str(hdr, 5)]
    count = 0
    for i in range(0, x_train.shape[0]):
        if count == maxcount: break
        count += 1
        disp.append([x_train[i, 0].astype(int),
                     x_train[i, 1].astype(int),
                     x_train[i, 2].astype(float),
                     *x_train[i, 3:].astype(float)
                    ])
    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow", floatfmt=flist, numalign='center')
    return table

In [7]:
def split_str(ifeatures, smax):
    ''' split the feature name strings to tables fit '''
    ofeatures = []
    for s in ifeatures:
        if not ' ' in s:  # skip string that already have a space
            if len(s) > smax:
                mid = int(len(s)/2)
                s = s[:mid] + " " + s[mid:]
        ofeatures.append(s)
    return ofeatures

In [8]:
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

,[use rid],[ratin gcount],[rati ngave],Pestisida & Obat Pertanian,Produk Pertanian Segar,Sarana dan Peralatan Irigasi,Produk Olahan Pertanian,Teknologi Pertanian Perlengkapan Rumah Kaca dan Pembibitan,Bahan Pengemasan dan Logistik,Bibit dan Benih Tanaman,Pupuk dan Nutrisi Tanaman,Alat dan Mesin Pertanian
1,250,3.5,4.1,3.4,3.8,3.0,3.6,3.7,3.5,3.7,3.4,3.2
1,250,3.5,4.1,3.4,3.8,3.0,3.6,3.7,3.5,3.7,3.4,3.2
1,250,3.5,4.1,3.4,3.8,3.0,3.6,3.7,3.5,3.7,3.4,3.2
1,250,3.5,4.1,3.4,3.8,3.0,3.6,3.7,3.5,3.7,3.4,3.2
1,250,3.5,4.1,3.4,3.8,3.0,3.6,3.7,3.5,3.7,3.4,3.2


In [9]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[prod uctid],rati ngave,Pestisida & Obat Pertanian,Produk Pertanian Segar,Sarana dan Peralatan Irigasi,Produk Olahan Pertanian,Teknologi Pertanian,Perlengkapan Rumah Kaca dan Pembibitan,Bahan Pengemasan dan Logistik,Bibit dan Benih Tanaman,Pupuk dan Nutrisi Tanaman,Alat dan Mesin Pertanian
1,3,1.0,0,0,0,0,0,0,0,0,0
1,3,1.0,0,0,0,0,0,0,0,0,0
1,3,1.0,0,0,0,0,0,0,0,0,0
5,3,1.0,0,0,0,0,0,0,0,0,0
5,3,1.0,0,0,0,0,0,0,0,0,0


In [10]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [5. 5. 5. 5. 5.]


In [11]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [12]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (139916, 12)
movie/item test data shape: (34980, 12)


In [13]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

,[use rid],[ratin gcount],[rati ngave],Pestisida & Obat Pertanian,Produk Pertanian Segar,Sarana dan Peralatan Irigasi,Produk Olahan Pertanian,Teknologi Pertanian Perlengkapan Rumah Kaca dan Pembibitan,Bahan Pengemasan dan Logistik,Bibit dan Benih Tanaman,Pupuk dan Nutrisi Tanaman,Alat dan Mesin Pertanian
0,0,-0.4,0.0,0.7,0.9,0.4,0.2,-0.4,0.4,0.0,-0.2,0.2
0,1,-0.2,0.2,0.3,0.0,0.1,0.5,0.7,0.2,0.6,0.1,-0.0
0,0,-0.6,0.9,0.0,0.2,-0.2,0.7,-1.2,0.1,0.7,0.6,0.0
1,0,0.7,-0.6,1.4,-2.9,-2.7,-2.2,-0.2,-1.9,1.5,1.4,0.0
0,0,-0.6,1.3,-0.3,1.5,0.3,-0.8,-3.0,0.1,-2.6,-0.2,-3.2


In [14]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs)
    ### END CODE HERE ###
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs)
    ### END CODE HERE ###
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=[num_user_features])
vu = user_NN(input_user)
vu = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vu)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features,))
vm = item_NN(input_item)
vm = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vm)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 11)]         0           []                               
                                                                                                  
 sequential (Sequential)        (None, 32)           39840       ['input_1[0][0]']                
                                                                                                  
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                 

In [15]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.CosineSimilarity()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [ ]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
4373/4373 [==============================] - 25s 5ms/step - loss: -0.2616
Epoch 2/30
4373/4373 [==============================] - 22s 5ms/step - loss: -0.2684
Epoch 3/30
4373/4373 [==============================] - 20s 5ms/step - loss: -0.2618
Epoch 4/30
4373/4373 [==============================] - 18s 4ms/step - loss: -0.2618
Epoch 5/30
4373/4373 [==============================] - 17s 4ms/step - loss: -0.2618
Epoch 6/30
 205/4373 [>.............................] - ETA: 13s - loss: -0.2401

In [ ]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

1094/1094 [==============================] - 9s 8ms/step - loss: -0.2671


-0.2670954763889313

In [ ]:
new_user_id = 8118
new_rating_ave = 0.0
new_peptisida = 5.0
new_pertaniansegar = 0.0
new_irigasi = 0.0
new_olahanpertanian = 5.0
new_teknologipertanian = 0.0
new_rumahkaca = 0.0
new_pengemasanlogistik = 10.0
new_bibitdanbenih = 15.0
new_pupuknutrisi = 100.0
new_alatmesin = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_peptisida, new_pertaniansegar, new_irigasi, new_olahanpertanian,
                      new_teknologipertanian, new_rumahkaca, new_pengemasanlogistik,
                      new_bibitdanbenih, new_pupuknutrisi, new_alatmesin]])

In [ ]:
def gen_user_vecs(user_vec, num_items):
    """ given a user vector return:
        user predict maxtrix to match the size of item_vecs """
    user_vecs = np.tile(user_vec, (num_items, 1))
    return user_vecs

# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# Check for duplicates in item_vecs
unique_items = np.unique(item_vecs, axis=0)
if len(unique_items) < len(item_vecs):
    print("Duplicate products found in item_vecs!")

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display


Duplicate products found in item_vecs!
5466/5466 [==============================] - 9s 2ms/step


In [ ]:
def print_pred_movies(y_p, item, movie_dict, maxcount=11):
    """ print results of prediction of a new user. inputs are expected to be in
        sorted order, unscaled. """
    count = 0
    disp = [["y_p", "productid", "ratingave", "productname", "category"]]

    for i in range(0, y_p.shape[0]):
        if count == maxcount:
            break
        count += 1
        movie_id = item[i, 0].astype(int)
        disp.append([np.around(y_p[i, 0], 1), item[i, 0].astype(int), np.around(item[i, 2].astype(float), 1),
                     movie_dict[movie_id]['productname'], movie_dict[movie_id]['category']])

    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
    return table

In [ ]:
print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 11)

y_p,productid,ratingave,productname,category
5,55,0,"Minyak Kelapa Ikan Dorang Spesial 4 pcs x 450ml (Total 1,8L 1800 ml)",Produk Olahan Pertanian
5,55,0,"Minyak Kelapa Ikan Dorang Spesial 4 pcs x 450ml (Total 1,8L 1800 ml)",Produk Olahan Pertanian
5,55,0,"Minyak Kelapa Ikan Dorang Spesial 4 pcs x 450ml (Total 1,8L 1800 ml)",Produk Olahan Pertanian
5,55,0,"Minyak Kelapa Ikan Dorang Spesial 4 pcs x 450ml (Total 1,8L 1800 ml)",Produk Olahan Pertanian
5,55,0,"Minyak Kelapa Ikan Dorang Spesial 4 pcs x 450ml (Total 1,8L 1800 ml)",Produk Olahan Pertanian
5,55,0,"Minyak Kelapa Ikan Dorang Spesial 4 pcs x 450ml (Total 1,8L 1800 ml)",Produk Olahan Pertanian
5,55,0,"Minyak Kelapa Ikan Dorang Spesial 4 pcs x 450ml (Total 1,8L 1800 ml)",Produk Olahan Pertanian
5,55,0,"Minyak Kelapa Ikan Dorang Spesial 4 pcs x 450ml (Total 1,8L 1800 ml)",Produk Olahan Pertanian
5,55,0,"Minyak Kelapa Ikan Dorang Spesial 4 pcs x 450ml (Total 1,8L 1800 ml)",Produk Olahan Pertanian
5,55,0,"Minyak Kelapa Ikan Dorang Spesial 4 pcs x 450ml (Total 1,8L 1800 ml)",Produk Olahan Pertanian


In [ ]:
def gen_user_vecs(user_vec, num_items):
    """ given a user vector return:
        user predict maxtrix to match the size of item_vecs """
    user_vecs = np.tile(user_vec, (num_items, 1))
    return user_vecs

In [ ]:
# predict on  everything, filter on print/use
def predict_uservec(user_vecs, item_vecs, model, u_s, i_s, scaler):
    """ given a scaled user vector, does the prediction on all movies in scaled print_item_vecs returns
        an array predictions sorted by predicted rating,
        arrays of user and item, sorted by predicted rating sorting index
    """
    y_p = model.predict([user_vecs[:, u_s:], item_vecs[:, i_s:]])
    y_pu = scaler.inverse_transform(y_p)

    if np.any(y_pu < 0):
        print("Error, expected all positive predictions")
    sorted_index = np.argsort(-y_pu, axis=0).reshape(-1).tolist()  #negate to get largest rating first
    sorted_ypu   = y_pu[sorted_index]
    sorted_items = item_vecs[sorted_index]
    sorted_user  = user_vecs[sorted_index]
    return(sorted_index, sorted_ypu, sorted_items, sorted_user)

In [ ]:
def get_user_vecs(user_id, user_train, item_vecs, user_to_genre):
    """ given a user_id, return:
        user train/predict matrix to match the size of item_vecs
        y vector with ratings for all rated movies and 0 for others of size item_vecs """

    if not user_id in user_to_genre:
        print("error: unknown user id")
        return None
    else:
        user_vec_found = False
        for i in range(len(user_train)):
            if user_train[i, 0] == user_id:
                user_vec = user_train[i]
                user_vec_found = True
                break
        if not user_vec_found:
            print("error in get_user_vecs, did not find uid in user_train")
        num_items = len(item_vecs)
        user_vecs = np.tile(user_vec, (num_items, 1))

        y = np.zeros(num_items)
        for i in range(num_items):  # walk through movies in item_vecs and get the movies, see if user has rated them
            movie_id = item_vecs[i, 0]
            if movie_id in user_to_genre[user_id]['movies']:
                rating = user_to_genre[user_id]['movies'][movie_id]
            else:
                rating = 0
            y[i] = rating
    return(user_vecs, y)

In [ ]:
# Load user_to_genre from a JSON file (if applicable)
with open('user_to_genre.pickle', 'rb') as f:  # Changed 'r' to 'rb' to open in binary mode
    user_to_genre = pickle.load(f)

uid = 2
# form a set of user vectors. This is the same vector, transformed and repeated.
user_vecs, y_vecs = get_user_vecs(uid, user_train_unscaled, item_vecs, user_to_genre)

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display
sorted_user  = user_vecs[sorted_index]
sorted_y     = y_vecs[sorted_index]



FileNotFoundError: [Errno 2] No such file or directory: 'user_to_genre.pickle'

In [ ]:
def print_existing_user(y_p, y, user, items, ivs, uvs, movie_dict, maxcount=10):
    """ print results of prediction for a user who was in the database.
        Inputs are expected to be in sorted order, unscaled.
    """
    count = 0
    disp = [["y_p", "y", "user", "user genre ave", "movie rating ave", "movie id", "title", "genres"]]
    count = 0
    for i in range(0, y.shape[0]):
        if y[i, 0] != 0:  # zero means not rated
            if count == maxcount:
                break
            count += 1
            movie_id = items[i, 0].astype(int)

            offsets = np.nonzero(items[i, ivs:] == 1)[0]
            genre_ratings = user[i, uvs + offsets]
            disp.append([y_p[i, 0], y[i, 0],
                         user[i, 0].astype(int),      # userid
                         np.array2string(genre_ratings,
                                         formatter={'float_kind':lambda x: "%.1f" % x},
                                         separator=',', suppress_small=True),
                         items[i, 2].astype(float),    # movie average rating
                         movie_id,
                         movie_dict[movie_id]['title'],
                         movie_dict[movie_id]['genres']])

    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow", floatfmt=[".1f", ".1f", ".0f", ".2f", ".1f"])
    return table

In [ ]:
#print sorted predictions for movies rated by the user
print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, ivs, uvs, movie_dict, maxcount = 50)

'<table>\n<thead>\n<tr></tr>\n</thead>\n<tbody>\n</tbody>\n</table>'

In [ ]:
# GRADED_FUNCTION: sq_dist
# UNQ_C2
def sq_dist(a,b):
    """
    Returns the squared distance between two vectors
    Args:
      a (ndarray (n,)): vector with n features
      b (ndarray (n,)): vector with n features
    Returns:
      d (float) : distance
    """
    ### START CODE HERE ###
    d = np.sum((a - b) ** 2)
    ### END CODE HERE ###
    return d

In [ ]:
a1 = np.array([1.0, 2.0, 3.0]); b1 = np.array([1.0, 2.0, 3.0])
a2 = np.array([1.1, 2.1, 3.1]); b2 = np.array([1.0, 2.0, 3.0])
a3 = np.array([0, 1, 0]);       b3 = np.array([1, 0, 0])
print(f"squared distance between a1 and b1: {sq_dist(a1, b1):0.3f}")
print(f"squared distance between a2 and b2: {sq_dist(a2, b2):0.3f}")
print(f"squared distance between a3 and b3: {sq_dist(a3, b3):0.3f}")

squared distance between a1 and b1: 0.000
squared distance between a2 and b2: 0.030
squared distance between a3 and b3: 2.000


In [ ]:
class NormalizeLayer(tf.keras.layers.Layer):
    def __init__(self, axis=1, **kwargs):
        super(NormalizeLayer, self).__init__(**kwargs)
        self.axis = axis

    def call(self, inputs):
        return tf.linalg.l2_normalize(inputs, axis=self.axis)

input_item_m = tf.keras.layers.Input(shape=(num_item_features,))    # input layer
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = NormalizeLayer(axis=1)(vm_m)                                # incorporate normalization as was done in the original model
model_m = tf.keras.Model(input_item_m, vm_m)
model_m.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 11)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 32)                  │          40,096 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ normalize_layer (NormalizeLayer)     │ (None, 32)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40,096 (156.62 KB)

 Trainable params: 40,096 (156.62 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

5466/5466 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step
size of all predicted movie feature vectors: (174896, 32)


In [ ]:
count = 50  # number of movies to display
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])

m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["productname1", "category", "productname2", "category"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    disp.append( [movie_dict[movie1_id]['title'], movie_dict[movie1_id]['genres'],
                  movie_dict[movie2_id]['title'], movie_dict[movie1_id]['genres']]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
table